### Create set of all object present in json files inside dedicated folder.

In [ ]:
import os
import json
folder = "../lineage_outputs/"
json_list = []
error_json_reading = dict()
for file in os.listdir(folder):
    with open(folder+file, "r") as f:
        json_file = json.load(f)
        json_file["file_name"] = file.replace(".json", "").split("--")[1] + "." + file.replace(".json", "").split("--")[2]
        try:
            src_trg_set = set()
            for src_trg_pairs in json_file["lineage"]:
                src_trg_set.add(src_trg_pairs["source"])
                src_trg_set.add(src_trg_pairs["target"])
            src_trg_set.remove(json_file["file_name"])
        except Exception as e:
            error_json_reading[json_file["file_name"]] = e
        
        json_file["content_set"] = src_trg_set
        json_list.append(json_file)

In [ ]:
print("error_json_reading: ", len(error_json_reading))
print("Key template of each dictionary in list: ",json_list[0].keys())
print(json_list)

### Create set of all lineage objects present in csv file for each dependent object.

In [ ]:
import pandas as pd
import re
# Load data
df_dev_linage = pd.read_csv("databases_lineage_extracted.csv")
df_uat_linage = pd.read_csv("database_lineage_extracted_2.csv")
df_unioned = pd.concat([df_dev_linage, df_uat_linage], ignore_index=True)
df_unioned

subset_cols = ['Dependent_Schema', 'Dependent_Object_Name', 'Dependent_Object_Type',
               'Depends_On_Schema', 'Depends_On_Object_Name', 'Depends_On_Object_Type']
df = df_unioned.drop_duplicates(subset=subset_cols)

In [ ]:
lineage_list = {}
for (dependent_schema, dependent_object_name), group in df.groupby(["Dependent_Schema","Dependent_Object_Name"]):
    dependent_object_name = re.sub(r"[^\w\s-]", "", dependent_object_name).replace(" ", "_")
    lineage_list[dependent_schema + "." + dependent_object_name] = set()
    for row in group.itertuples():
        content = row[7] + '.' + row[8]
        lineage_list[dependent_schema + "." + dependent_object_name].add(content)

### Analyze both data structure and find out unequal sets containing lineage objects

In [ ]:
correct = []
incorrect = []
errors = []
for json_item in json_list:
    try:
        if json_item["content_set"] == lineage_list[json_item["file_name"]]:
            correct.append(json_item["file_name"])
        else:
            incorrect.append(json_item["file_name"])
    except Exception as e:
        errors.append(e)

In [ ]:
print("correct_jsons: " ,len(correct), "\nIncorrect_jsons: ", len(incorrect), "\nTotal jsons processed :", len(correct) + len(incorrect), "\nErrors: ", len(errors))

In [ ]:
for object in incorrect:
    print(object)

In [ ]:
# for object in incorrect:
#     if object == "tmp.usp_Cobachi_Business_Hierarchy_PST":
#         print(True)
#     else:
#         continue

## Search content from csv files

In [ ]:
#####################
depend_schema = ""
dependent_object_name = ""
#####################

In [ ]:
## Find item and it's lineage set in json_list read from json files.
for item in json_list:
    if item['file_name'] == depend_schema + '.' + dependent_object_name:
        print("file_name present in json_list\n")
        print(item['content_set'])

In [ ]:
for item in lineage_list:
    if item == depend_schema + '.' + dependent_object_name:
        print('Dependent_object_name: ', item, "\n Lineage set: ", lineage_list[item])

In [ ]:
df_dev_linage = pd.read_csv("databases_lineage_extracted.csv")
df_uat_linage = pd.read_csv("database_lineage_extracted_2.csv")
df_unioned = pd.concat([df_dev_linage, df_uat_linage], ignore_index=True)
df_unioned

subset_cols = ['Dependent_Schema', 'Dependent_Object_Name', 'Dependent_Object_Type',
               'Depends_On_Schema', 'Depends_On_Object_Name', 'Depends_On_Object_Type']
df = df_unioned.drop_duplicates(subset=subset_cols)
df = df.query(f"Dependent_Schema == '{depend_schema}' and Dependent_Object_Name == '{dependent_object_name}'")
df

In [ ]:
def robust_clean_sql(sql_query):
    sql_text = str(sql_query)

    sql_text = sql_text.replace('\\n', '\n').replace('\\t', '\t')

    # Remove single-line comments (-- ...)
    sql_text = re.sub(r'--.*', '', sql_text)
    # Remove multi-line comments (/* ... */)
    sql_text = re.sub(r'/\*.*?\*/', '', sql_text, flags=re.DOTALL)

    # Replace multiple newlines with a single newline
    sql_text = re.sub(r'\n\s*\n', '\n', sql_text)
    # Collapse horizontal spaces (tabs/spaces) into one space
    sql_text = re.sub(r'[ \t]+', ' ', sql_text)
    
    return sql_text.strip()

df_definitions = pd.concat([pd.read_csv("object_definitions.csv"), pd.read_csv("UAT_object_definitions.csv")], ignore_index=True)
df_definitions = df_definitions.query(f"Schema == '{depend_schema}' and Object == '{dependent_object_name}'")
df_definitions

In [ ]:
print(robust_clean_sql(df_definitions.iloc[0,4]))